<a href="https://colab.research.google.com/github/itsMatrik/fish-audio-google-colab-notebook/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fish Speech

# **READ IT PLEASE**

This is fork from original fish-speech (https://github.com/fishaudio/fish-speech). This is google colab notebook version because original repo dont have it. Fully working and ready to use. Just run commands line-by-line. You can get hugginfFace token here: https://huggingface.co/settings/tokens (first you need to create an account and subscribe on s1-mini model on huggingFace model page (its free)). If something is not working chatGPT is free

### For Windows User / win用户

In [ ]:
!chcp 65001

### For Linux User / Linux 用户

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

In [ ]:
!pip install -q huggingface_hub
import huggingface_hub
print("huggingface_hub version:", huggingface_hub.__version__)

In [ ]:
from getpass import getpass
from huggingface_hub import login
hf_token = getpass("insert huggingFace token: ")
login(token=hf_token)

In [ ]:
!git clone https://github.com/fishaudio/fish-speech.git

In [ ]:
%cd fish-speech

In [ ]:
!apt-get install -y portaudio19-dev

In [ ]:
!pip install -e .

In [ ]:
!python -c "import sys; import torch, torchaudio; print('python', sys.version.split()[0]); print('torch', getattr(torch,'__version__',None)); print('torchaudio', getattr(torchaudio,'__version__',None))"

In [ ]:
!pip uninstall -y torchaudio torch

In [ ]:
!pip install --force-reinstall torch==2.6.0+cu124 torchaudio==2.6.0+cu124 --index-url https://download.pytorch.org/whl/cu124

In [ ]:
!pip uninstall -y torchvision torchmetrics

In [ ]:
!pip install --force-reinstall torchvision==0.21.0+cu124 --index-url https://download.pytorch.org/whl/cu124

In [ ]:
!pip install --no-cache-dir --upgrade "torchmetrics==1.8.2"

### Prepare Model

In [ ]:
# For Chinese users, you probably want to use mirror to accelerate downloading
# !set HF_ENDPOINT=https://hf-mirror.com
# !export HF_ENDPOINT=https://hf-mirror.com

!hf download fishaudio/openaudio-s1-mini --local-dir checkpoints/openaudio-s1-mini/

## WebUI Inference

> You can use --compile to fuse CUDA kernels for faster inference (10x).

In [ ]:
import os
os.environ['GRADIO_SHARE'] = 'True'

!python tools/run_webui.py \
  --llama-checkpoint-path checkpoints/openaudio-s1-mini \
  --decoder-checkpoint-path checkpoints/openaudio-s1-mini/codec.pth \
  --device cuda \
  --half \
  --compile

## Break-down CLI Inference

### 1. Encode reference audio: / 从语音生成 prompt:

You should get a `fake.npy` file.

你应该能得到一个 `fake.npy` 文件.

In [ ]:
## Enter the path to the audio file here
src_audio = r"D:\PythonProject\vo_hutao_draw_appear.wav"

!python fish_speech/models/dac/inference.py \
    -i {src_audio} \
    --checkpoint-path "checkpoints/openaudio-s1-mini/codec.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)

### 2. Generate semantic tokens from text: / 从文本生成语义 token:

> This command will create a codes_N file in the working directory, where N is an integer starting from 0.

> You may want to use `--compile` to fuse CUDA kernels for faster inference (~30 tokens/second -> ~300 tokens/second).

> 该命令会在工作目录下创建 codes_N 文件, 其中 N 是从 0 开始的整数.

> 您可以使用 `--compile` 来融合 cuda 内核以实现更快的推理 (~30 tokens/秒 -> ~300 tokens/秒)

In [ ]:
!python fish_speech/models/text2semantic/inference.py \
    --text "hello world" \
    --prompt-text "The text corresponding to reference audio" \
    --prompt-tokens "fake.npy" \
    --checkpoint-path "checkpoints/openaudio-s1-mini" \
    --num-samples 2
    # --compile

### 3. Generate speech from semantic tokens: / 从语义 token 生成人声:

In [ ]:
!python fish_speech/models/dac/inference.py \
    -i "codes_0.npy" \
    --checkpoint-path "checkpoints/openaudio-s1-mini/codec.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)